In [1]:
#!pip install pandas==1.3.5

In [2]:
import pandas as pd
import sys
import os 

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import string
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType
import warnings

from recommenders.utils.spark_utils import start_or_get_spark
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder


warnings.simplefilter(action='ignore', category=FutureWarning)

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Spark version: 3.1.2


In [3]:
pd.__version__

'1.3.5'

In [22]:
ratings_data = pd.read_csv(
        r"C:\Users\mirza\Desktop\dataset\MovieLens-1M\ratings.csv",sep = "::",
        names=["userId", "movieId", "ratings", "timestamp"])

movie= pd.read_csv(
        r"C:\Users\mirza\Desktop\dataset\MovieLens-1M\movies.csv",sep = "::",
        names=["movieId", "Title", "Genres"])

C:\Users\mirza\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [5]:
 spark = SparkSession \
  .builder \
  .appName("Example") \
  .config("spark.driver.memory", "16g")\
  .getOrCreate()

In [6]:
ratings_data=spark.createDataFrame(ratings_data) 
#movie=spark.createDataFrame(movie) 

In [7]:
for field in ratings_data.schema.fields:
    print(field.name +" , "+str(field.dataType))

userId , LongType
movieId , LongType
ratings , LongType
timestamp , LongType


In [8]:
(training, test) = ratings_data.randomSplit ([0.8, 0.2])

als = ALS(userCol="userId", itemCol="movieId", ratingCol="ratings", coldStartStrategy="drop", nonnegative=True)

In [9]:
param_grid = ParamGridBuilder()\
            .addGrid(als.rank, [10])\
            .addGrid(als.maxIter, [15])\
            .addGrid(als.regParam, [0.05])\
            .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="ratings",
                               predictionCol="prediction")

tvs = TrainValidationSplit(estimator = als,
                          estimatorParamMaps= param_grid,
                          evaluator = evaluator)


In [10]:
model = tvs.fit(training)

best_model = model.bestModel

predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

print("RMSE = " + str(rmse))
print("**Best Model**")
print(" Rank:"), best_model.rank
print(" MaxIter:"), best_model._java_obj.parent().getMaxIter()
print(" RegParam:"), best_model._java_obj.parent().getRegParam()

RMSE = 0.8564721754793594
**Best Model**
 Rank:
 MaxIter:
 RegParam:


(None, 0.05)

In [11]:
user_recs = best_model.recommendForAllUsers(5)

In [12]:
user_recs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{572, 5.062233},...|
|    27|[{2503, 5.1979637...|
|    28|[{3310, 4.730515}...|
|    31|[{572, 5.1795774}...|
|    34|[{572, 5.877676},...|
|    44|[{214, 5.0411854}...|
|    53|[{557, 6.218903},...|
|    65|[{572, 6.2710576}...|
|    76|[{572, 5.6091995}...|
|    78|[{666, 4.874102},...|
|    81|[{2323, 5.728367}...|
|    85|[{2964, 6.045653}...|
|   101|[{1421, 7.050477}...|
|   108|[{2494, 5.1378493...|
|   115|[{1420, 5.465139}...|
|   126|[{3192, 5.730672}...|
|   133|[{572, 5.963087},...|
|   137|[{572, 5.178694},...|
|   148|[{572, 5.5068383}...|
|   155|[{1519, 4.9232388...|
+------+--------------------+
only showing top 20 rows



In [13]:
user_recs.filter(user_recs.userId == "140").show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   140|[{2931, 4.8981066...|
+------+--------------------+



In [75]:
movie["movieId"]= movie["movieId"].str.replace('[{}]'.format(string.punctuation), '')
movie["movieId"]=movie["movieId"].astype(int)
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  3883 non-null   int32 
 1   Title    3883 non-null   object
 2   Genres   3883 non-null   object
dtypes: int32(1), object(2)
memory usage: 76.0+ KB


In [15]:
movie.drop(["Genres"],axis=1, inplace=True)

In [81]:
def get_recs_for_user(user):
    recs = user_recs.filter(user_recs.userId==user)
    recs = recs.select("recommendations.movieId", "recommendations.rating") # add "userId" ıf you want to add userId column
    movies = recs.select("movieId").toPandas().iloc[0,0]
    #ratings= recs.select("rating").toPandas().iloc[0,0]
    #userıd= recs.select("userId").toPandas().iloc[0,0] 
    ratings_matrix= pd.DataFrame(movies, columns=["movieId"])
    #ratings_matrix["ratings"]=ratings
    #ratings_matrix["userId"]=userıd  
    #ratings_matrix_ps= spark.createDataFrame(ratings_matrix) #ıf you want to get spark dataframe
    matrix = pd.merge(ratings_matrix, movie, how="inner", on="movieId")
    

    recommended_movies = []
    recommended_movies_posters = []
    
    for i in matrix.movieId:
        recommended_movies_posters.append(i)
        
    for k in matrix.Title:
        recommended_movies.append(k)
    
    return recommended_movies, recommended_movies_posters

In [82]:
get_recs_for_user(140)

(['Time of the Gypsies (Dom za vesanje) (1989)',
  'Zachariah (1971)',
  'I Am Cuba (Soy Cuba/Ya Kuba) (1964)',
  'World of Apu, The (Apur Sansar) (1959)',
  'Leather Jacket Love Story (1997)'],
 [2931, 3236, 3245, 670, 1851])

In [18]:
movie

,movieId,Title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
3878,3948,Meet the Parents (2000)
3879,3949,Requiem for a Dream (2000)
3880,3950,Tigerland (2000)
3881,3951,Two Family House (2000)


In [66]:
import pickle

pickle.dump(movie, open("movie.pkl","wb" ))
pickle.dump(users, open("users.pkl","wb" ))

In [ ]:
user_recs.select("userId", "recommendations").write.save("user_recs.parquet")